### BiRealNet
This notebook demonstrates the binarization of ResNet Model using the BiRealNet

In [ ]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [ ]:
import sys
import os
import shutil
sys.path.append("../../../")
sys.path.append("../../")

In [3]:
# load important packages
import torch
import torchvision
import matplotlib.pyplot as plt
from torchvision import transforms
from trailmet.datasets.classification import DatasetFactory

###  Define Model and Model Compression Strategy

In [4]:
from trailmet.algorithms.binarize.birealnet import BirealNet
from trailmet.models.resnet_bireal import make_birealnet50

In [5]:
data_root = './'

In [6]:
def set_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    os.environ['PYTHONHASHSEED'] = str(seed)
    print('> SEEDING DONE')
    
set_seed(1024)  

> SEEDING DONE


### Augmentations

In [7]:
# Augmentation
crop_scale = 0.08
lighting_param = 0.1
train_transform = transforms.Compose([
    transforms.RandomResizedCrop(32, scale=(crop_scale, 1.0)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor()])

valid_transform = transforms.Compose([
        transforms.Resize(36),
        transforms.CenterCrop(32),
        transforms.ToTensor()
    ])
test_transform = valid_transform

input_transforms = {
    'train': train_transform, 
    'val': valid_transform, 
    'test': test_transform}

target_transforms = {
    'train': None, 
    'val': None, 
    'test': None}

### Configuration Class

In [8]:
root_dir = './'
import yaml
with open(os.path.join(root_dir,"birealnet_cifar100.yaml"),'r') as stream:
    kwargs = yaml.safe_load(stream)
kwargs

{'batch_size': 256,
 'valid_size': 0.1,
 'num_train': 0,
 'epochs': 1,
 'optimizer': 'torch.optim.Adam',
 'lr': 0.001,
 'momentum': 0.9,
 'save_path': './save_path_resnet50_cifar100_fullbin',
 'data_path': '',
 'label_smooth': 0.1,
 'weight_decay': 0,
 'workers': 4,
 'device': 'cuda',
 'dataset': 'c100',
 'num_classes': 100,
 'num_fp': 0}

### Define DataLoaders

In [9]:
# Create Dataset
cifar100_dataset = DatasetFactory.create_dataset(name = 'CIFAR100', 
                                        root = root_dir,
                                        split_types = ['train', 'val', 'test'],
                                        val_fraction = 0.15,
                                        transform = input_transforms,
                                        target_transform = target_transforms
                                        )
# Define DataLoaders
train_loader100 = torch.utils.data.DataLoader(
        cifar100_dataset['train'], batch_size=kwargs['batch_size'], 
        sampler=cifar100_dataset['train_sampler'],
        num_workers=kwargs['workers']
    )
val_loader100 = torch.utils.data.DataLoader(
        cifar100_dataset['val'], batch_size=kwargs['batch_size'], 
        sampler=cifar100_dataset['val_sampler'],
        num_workers=kwargs['workers']
    )
test_loader100 = torch.utils.data.DataLoader(
        cifar100_dataset['test'], batch_size=kwargs['batch_size'], 
        sampler=cifar100_dataset['test_sampler'],
        num_workers=kwargs['workers']
    )

dataloaders = {
        'train': train_loader100, 'val': val_loader100, "test": test_loader100
}


Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


### Model

In [10]:
model = make_birealnet50(num_classes=kwargs['num_classes'], insize=32, num_fp=kwargs['num_fp'])

In [11]:
algo = BirealNet(model, dataloaders, **kwargs)

### Training and Evaluating the Binarized ResNet Model

In [12]:
algo.binarize()

03/19 01:28:38 PM CFG = {'batch_size': 256, 'valid_size': 0.1, 'num_train': 0, 'epochs': 1, 'optimizer': 'torch.optim.Adam', 'lr': 0.001, 'momentum': 0.9, 'save_path': './save_path_resnet50_cifar100_fullbin', 'data_path': '', 'label_smooth': 0.1, 'weight_decay': 0, 'workers': 4, 'device': 'cuda', 'dataset': 'c100', 'num_classes': 100, 'num_fp': 0}
03/19 01:28:38 PM BirealNet(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (activ): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BinaryBasicBlock(
      (activation): BinaryActivation()
      (conv): HardBinaryConv()
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BinaryBasicBlock(
      (activation): BinaryActivation()
      (conv): HardBinaryConv()
      (bn

/opt/conda/envs/ml/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:138: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "


Epoch: [0][  0/167]	Time  3.609 ( 3.609)	Data  0.594 ( 0.594)	Loss 4.7160e+00 (4.7160e+00)	Acc@1   1.17 (  1.17)	Acc@5   4.69 (  4.69)
Epoch: [0][  1/167]	Time  0.057 ( 1.833)	Data  0.003 ( 0.299)	Loss 4.7138e+00 (4.7149e+00)	Acc@1   1.17 (  1.17)	Acc@5   5.08 (  4.88)
Epoch: [0][  2/167]	Time  0.053 ( 1.239)	Data  0.004 ( 0.200)	Loss 4.7252e+00 (4.7183e+00)	Acc@1   1.17 (  1.17)	Acc@5   3.91 (  4.56)
Epoch: [0][  3/167]	Time  0.049 ( 0.942)	Data  0.003 ( 0.151)	Loss 4.7311e+00 (4.7215e+00)	Acc@1   1.56 (  1.27)	Acc@5   7.03 (  5.18)
Epoch: [0][  4/167]	Time  0.052 ( 0.764)	Data  0.005 ( 0.122)	Loss 4.7660e+00 (4.7304e+00)	Acc@1   1.17 (  1.25)	Acc@5   4.69 (  5.08)
Epoch: [0][  5/167]	Time  0.050 ( 0.645)	Data  0.003 ( 0.102)	Loss 4.7695e+00 (4.7369e+00)	Acc@1   1.17 (  1.24)	Acc@5   5.08 (  5.08)
Epoch: [0][  6/167]	Time  0.049 ( 0.560)	Data  0.004 ( 0.088)	Loss 4.7660e+00 (4.7411e+00)	Acc@1   0.39 (  1.12)	Acc@5   3.91 (  4.91)
Epoch: [0][  7/167]	Time  0.047 ( 0.496)	Data  0.003 ( 

### Loading the Checkpoint to know the test accuracy

In [14]:
chk=torch.load('./save_path_resnet50_cifar100_fullbin/model_best.pth.tar')

In [15]:
chk['epoch']

0

### Test Accuracy

In [16]:
chk['best_top1_acc']

tensor(0.9600, device='cuda:0')